In [1]:
import pandas as pd
import numpy as np
from numpy import linalg
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
import nltk
import pickle
from nltk import pos_tag
from scipy.sparse import hstack, vstack
from scipy import spatial
from sklearn.datasets import dump_svmlight_file,load_svmlight_file
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
import gensim
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import multiprocessing
import gc
import logging
import time
import os
# 显示cell运行时长
%load_ext klab-autotime

In [15]:
# ###################### 周周星 word2vec #################
# class dataGenerator():
#     def __init__(self,dirName):
#         self.dirName=dirName
#     def __iter__(self):
#         for fname in os.listdir(self.dirName):
#             for line in open(os.path.join(self.dirName,fname)):
#                 datas=line.split(',')
#                 yield datas[1].split(" ")+datas[3].split(" ")
                
# dir_name = "/home/kesci/input/bytedance/first-round/"
# vector_size = 128
# savedir = "/home/kesci/work/word2vec/"
# save_name = "word2vector.model"

# class EpochSaver(gensim.models.callbacks.CallbackAny2Vec):
#     '''用于保存模型, 打印损失函数等等'''
#     def __init__(self, savedir, save_name):
#         os.makedirs(savedir, exist_ok=True)
#         self.save_path = os.path.join(savedir, save_name)
#         self.epoch = 0
#         self.pre_loss = 0
#         self.best_loss = 999999999.9
#         self.since = time.time()

#     def on_epoch_end(self, model):
#         self.epoch += 1
#         cum_loss = model.get_latest_training_loss() # 返回的是从第一个epoch累计的
#         epoch_loss = cum_loss - self.pre_loss
#         time_taken = time.time() - self.since
#         print("Epoch %d, loss: %.2f, time: %dmin %ds" % 
#                     (self.epoch, epoch_loss, time_taken//60, time_taken%60))
#         if self.best_loss > epoch_loss:
#             self.best_loss = epoch_loss
#             print("Better model. Best loss: %.2f" % self.best_loss)
#             model.save(self.save_path)
#             print("Model %s save done!" % self.save_path)

#         self.pre_loss = cum_loss
#         self.since = time.time()

# # 将整个过程分成三步
# # 1, 构建模型(不训练)
# model_word2vec = gensim.models.Word2Vec(min_count=2, 
#                                         window=10, 
#                                         size=vector_size,
#                                         workers=63,
#                                         batch_words=100000)
# # 2, 遍历一遍语料库
# since = time.time()
# sentences = dataGenerator(dir_name) # 你的sentence迭代器
# model_word2vec.build_vocab(sentences, progress_per=20000000)
# time_elapsed = time.time() - since
# print('Time to build vocab: {:.0f}min {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

# # 3, 训练
# since = time.time()
# model_word2vec.train(sentences, total_examples=model_word2vec.corpus_count, 
#                         epochs=8, compute_loss=True, report_delay=60*10, # 每隔10分钟输出一下日志
#                         callbacks=[EpochSaver(savedir, save_name)])
# time_elapsed = time.time() - since
# print('Time to train: {:.0f}min {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

time: 4.12 ms


In [2]:
train_data = "/home/kesci/input/bytedance/first-round/train.csv"
test_data = "/home/kesci/input/bytedance/first-round/test.csv"
model_path = "/home/kesci/work/word2vec/"
colnames = ["query_id","query","query_title_id","title","label"]
min_count = 2
window = 10
word_dim = 128
w2viter = 10
cores = multiprocessing.cpu_count()-1

time: 1.26 ms


In [3]:
corpus_file = train_data
chunksize = 10000
class getSentences:
    def __init__(self, filename_list):
        self.filename_list = filename_list
    def __iter__(self):
        for filename in self.filename_list:
            for df in pd.read_csv(filename,names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
                for row in df.values:
                    yield  row[1].split(" ") +  row[3].split(" ") 
corpus_file_list = [train_data,test_data]
sentences = getSentences(corpus_file_list)


time: 2.85 ms


In [4]:
logging.basicConfig(filename="/home/kesci/word2vec.log",filemode="w",format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
###### 6.15 重新训练word2vec
w2vmodel = gensim.models.Word2Vec(sentences,window=window,size=word_dim,workers=cores,min_count=min_count,iter=w2viter)    
pickle.dump(w2vmodel,open(model_path+"word2vec_model.pkl","wb"))


time: 4h 40min 23s


In [4]:
############## 加入测试集的训练 word2vec ##############
############# 已完成 ############
w2vmodel = pickle.load(open(model_path+"word2vec_model_new.pkl","rb"))
corpus_file = "/home/kesci/input/bytedance/first-round/test.csv"
sentences_test = getSentences(corpus_file)
w2vmodel.build_vocab(sentences_test, update=True)
w2vmodel.train(sentences_test,total_examples=w2vmodel.corpus_count,epochs=w2vmodel.epochs)
pickle.dump(w2vmodel,open(model_path+"word2vec_model_new_withTest.pkl","wb"))  

time: 642 µs


In [6]:
####### 查看日志
# !cat /home/kesci/word2vec.log | tail -n 10

time: 531 µs


In [7]:
model_path = "/home/kesci/work/word2vec/"
w2vmodel = pickle.load(open(model_path+"word2vec_model_new.pkl","rb"))
w2vmodel_test = pickle.load(open(model_path+"word2vec_model_new_withTest.pkl","rb"))

time: 8.31 s


In [8]:
###### 测试集中有较多的新单词 ######
###### 测试集中有较多的新单词 ######
###### 测试集中有较多的新单词 ######
print(len(w2vmodel.wv.vocab))
print(len(w2vmodel_test.wv.vocab))


842336
979898
time: 1.35 ms


In [17]:
########### 对比若干单词的相似度 ##############
### 以前的增量训练 导致相似度较低(也有可能是以前的window size设置的较大  or 提高了特征维度大小)  ###
from scipy import spatial
1 - spatial.distance.cosine(w2vmodel.wv["1427"], w2vmodel_test.wv["1427"])


0.9197492003440857

time: 4.44 ms


In [21]:
######### 对于加入test测试集的w2v 保存dict ######
from gensim.models import KeyedVectors
# # save dict
# save_path = "/home/kesci/work/word2vec/wordvectors.kv"
# w2vmodel_test = pickle.load(open(model_path+"word2vec_model_new_withTest.pkl","rb"))
# w2vmodel_test.wv.save(save_path)

# 测试
save_path = "/home/kesci/work/word2vec/wordvectors.kv"
w2v_dict = KeyedVectors.load(save_path, mmap='r')
len(w2v_dict["1427"])

128

time: 3.47 s


In [6]:
################### 2019.06.15 重跑similarity特征  ###############################
################### 2019.06.15 重跑similarity特征  ###############################
################### 2019.06.15 重跑similarity特征   ###############################
################### 2019.06.15 重跑similarity特征  ###############################
################### 2019.06.15 重跑similarity特征  ###############################
################### 2019.06.15 重跑similarity特征  ###############################

time: 1.02 ms


In [11]:
len(w2vmodel.wv.vocab)

855706

time: 10.4 ms


In [13]:
colnames = ["query_id","query","query_title_id","title","label"]
# save w2v dict
save_path = "/home/kesci/work/word2vec/wordvectors.kv"
w2vmodel_test = pickle.load(open(model_path+"word2vec_model.pkl","rb"))
w2vmodel_test.wv.save(save_path)
save_path = "/home/kesci/work/word2vec/wordvectors.kv"
w2v_dict = KeyedVectors.load(save_path, mmap='r')
len(w2v_dict["1427"])
# load idf——weight
q_idf_weight = pickle.load(open("/home/kesci/work/idf/idf_weight_query.pkl","rb"))
t_idf_weight = pickle.load(open("/home/kesci/work/idf/idf_weight_title.pkl","rb"))


time: 13.8 s


In [14]:
def cosine_sim(query, title):
    l2_query = linalg.norm(query,ord=2,axis=1)
    l2_title = linalg.norm(title,ord=2,axis=1)
    eudistance = linalg.norm(query-title,ord=2,axis=1) # 欧式距离 L2
    manhattan = linalg.norm(query-title,ord=1,axis=1) # 曼哈顿距离 L1
    chebyshev = linalg.norm(query-title,ord=np.inf,axis=1) # 切比雪夫距离 Max|x1-x2|
    chebyshev_ = linalg.norm(query-title,ord=-np.inf,axis=1) # Min |x1-x2|
    my_sqrt_distance = np.sum(np.sqrt(np.absolute(query-title)),axis=1) # sqrt|x1-x2|
    sim = np.sum(query * title,axis=1)/(l2_query * l2_title)
    
    combine_sim = np.hstack((sim.reshape(-1,1),
                             eudistance.reshape(-1,1),
                             manhattan.reshape(-1,1),
                             chebyshev.reshape(-1,1),chebyshev_.reshape(-1,1),
                             my_sqrt_distance.reshape(-1,1)
                            ))
    return combine_sim

#### 1w大小的数据  计算一次
def concat_sentence(origin_data):
    q_vstack = ""
    t_vstack = ""
    for row in origin_data.values:
        # 1.query
        words = row[1].split(" ")
        M = []
        for w in words:
            try:
                M.append(w2v_dict[w] * q_idf_weight[w])
            except:
                continue
        M = np.array(M)
        if M.shape[0] != 0:
            q_sentence_vec = M.sum(axis=0) / M.shape[0]
        else:
            q_sentence_vec = np.full(word_dim,np.finfo(np.float32).eps)
        if q_vstack == "":
            q_vstack = q_sentence_vec
        else:
             q_vstack = np.vstack((q_vstack,q_sentence_vec))
        
        # 2.title
        words = row[3].split(" ")
        M = []
        for w in words:
            try:
                M.append(w2v_dict[w] * t_idf_weight[w])
            except:
                continue
        M = np.array(M)
        if M.shape[0] != 0:
            t_sentence_vec = M.sum(axis=0) / M.shape[0]
        else:
            t_sentence_vec = np.full(word_dim,np.finfo(np.float32).eps)
        if t_vstack == "":
            t_vstack = t_sentence_vec
        else:
             t_vstack = np.vstack((t_vstack,t_sentence_vec))
    return q_vstack,t_vstack

time: 8.45 ms


In [15]:
############## 后500w 训练集数据 ############
feature_path = "/home/kesci/work/similarity_feature/train_idf/"
train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# sentece vector of (query-title)
sentence_vector_path = feature_path+"sentence_vector.csv"
sentence_vec_fout = open(sentence_vector_path,"w")
# 6dim similarity feature
consine_feature_path = feature_path+"consine_sentence_sim.csv"
sim_qt_fout = open(consine_feature_path,"w")

samples = 100000000
chunksize = 10000
skip_num = int(samples/chunksize) - 500
print(skip_num)

for i in range(int(500)):
    df = pd.read_csv(train_data, names=colnames, header=None,skiprows=chunksize*(skip_num+i),nrows=chunksize,lineterminator="\n")
    # 1.get sentece_vector & consine sim
    q_vstack,t_vstack = concat_sentence(df)
    combine_sim = cosine_sim(q_vstack,t_vstack)
    # 2.save
    np.savetxt(sentence_vec_fout,q_vstack - t_vstack,delimiter=",",fmt="%.5f")
    np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f")
    # 3.print log
    if (i+1) % 100 == 0:
        print((q_vstack - t_vstack).shape,combine_sim.shape)
    # # 4.gc
    # del df,q_vstack,t_vstack
    # gc.collect()

sentence_vec_fout.flush()
sim_qt_fout.flush()


9500


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
time: 5h 20min 6s


In [16]:
!cat /home/kesci/work/similarity_feature/train_idf/consine_sentence_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/train_idf/sentence_vector.csv | wc -l
!cat /home/kesci/work/similarity_feature/train_idf/consine_sentence_sim.csv | head -n 1
!cat /home/kesci/work/similarity_feature/train_idf/sentence_vector.csv | head -n 1

5000000
5000000
0.78677,67.51521,626.75317,14.74798,0.00117,263.97629
cat: write error: Broken pipe
-11.87070,1.43505,-3.70975,-2.85847,-0.23979,-4.70005,-0.00117,-4.35268,-1.77644,8.38602,2.08615,-1.11998,-1.79297,-1.80242,-3.28345,-1.63598,14.74798,5.47955,4.49442,-1.04792,-9.18590,-4.25819,8.44214,7.78709,2.45657,-10.49086,2.80651,-3.31167,2.09307,13.47444,-8.11537,10.25090,-10.84212,12.15462,7.54170,6.33480,-7.32336,-8.45119,8.37578,-5.19715,-0.93039,12.26204,-4.55682,5.84306,-7.00140,0.48155,-4.28800,-3.39937,-5.18955,5.17020,12.94971,-1.39133,-9.43076,0.46021,-1.12013,4.04392,7.16077,-4.52341,7.80053,1.78928,-5.12853,-11.04472,-4.28940,-3.29738,-5.05473,8.37624,13.25131,2.11399,-9.89016,10.99683,6.59390,4.24062,3.71435,1.03263,2.60437,7.22367,-1.98669,-2.07145,-2.66088,4.19020,2.62542,7.46982,1.19146,2.69422,0.31037,3.40124,-4.53363,0.82633,5.69024,-1.76573,-8.01711,-2.39459,0.86877,-7.27101,7.54878,-1.31771,-1.09998,-2.39850,3.23776,-4.21766,7.43775,-3.62145,-6.31628,1.28724,3.6

In [ ]:
############## 1000w-500w 验证集数据 ############
feature_path = "/home/kesci/work/similarity_feature/valid_idf/"
train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# sentece vector of (query-title)
sentence_vector_path = feature_path+"sentence_vector.csv"
sentence_vec_fout = open(sentence_vector_path,"w")
# 6dim similarity feature
consine_feature_path = feature_path+"consine_sentence_sim.csv"
sim_qt_fout = open(consine_feature_path,"w")

samples = 100000000
chunksize = 10000
skip_num = int(samples/chunksize) - 1000
print(skip_num)

for i in range(int(500)):
    df = pd.read_csv(train_data, names=colnames, header=None,skiprows=chunksize*(skip_num+i),nrows=chunksize,lineterminator="\n")
    # 1.get sentece_vector & consine sim
    q_vstack,t_vstack = concat_sentence(df)
    combine_sim = cosine_sim(q_vstack,t_vstack)
    # 2.save
    np.savetxt(sentence_vec_fout,q_vstack - t_vstack,delimiter=",",fmt="%.5f")
    np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f")

    # 3.print log
    if (i+1) % 100 == 0:
        print((q_vstack - t_vstack).shape,combine_sim.shape)
    # # 4.gc
    # del df,q_vstack,t_vstack
    # gc.collect()

sentence_vec_fout.flush()
sim_qt_fout.flush()


In [ ]:
!cat /home/kesci/work/similarity_feature/valid_idf/consine_sentence_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/valid_idf/sentence_vector.csv | wc -l
!cat /home/kesci/work/similarity_feature/valid_idf/consine_sentence_sim.csv | head -n 1
!cat /home/kesci/work/similarity_feature/valid_idf/sentence_vector.csv | head -n 1

In [ ]:
############## 测试集数据 ############
feature_path = "/home/kesci/work/similarity_feature/test_idf/"
test_data = "/home/kesci/input/bytedance/first-round/test.csv"
# sentece vector of (query-title)
sentence_vector_path = feature_path+"sentence_vector.csv"
sentence_vec_fout = open(sentence_vector_path,"w")
# 6dim similarity feature
consine_feature_path = feature_path+"consine_sentence_sim.csv"
sim_qt_fout = open(consine_feature_path,"w")

chunksize = 10000

for i in range(int(500)):
    df = pd.read_csv(test_data, names=colnames, header=None,skiprows=chunksize*i,nrows=chunksize,lineterminator="\n")
    # 1.get sentece_vector & consine sim
    q_vstack,t_vstack = concat_sentence(df)
    combine_sim = cosine_sim(q_vstack,t_vstack)
    # 2.save
    np.savetxt(sentence_vec_fout,q_vstack - t_vstack,delimiter=",",fmt="%.5f")
    np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f")

    # 3.print log
    if (i+1) % 100 == 0:
        print((q_vstack - t_vstack).shape,combine_sim.shape)
    # # 4.gc
    # del df,q_vstack,t_vstack
    # gc.collect()

sentence_vec_fout.flush()
sim_qt_fout.flush()


In [ ]:
!cat /home/kesci/work/similarity_feature/test_idf/consine_sentence_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/test_idf/sentence_vector.csv | wc -l
!cat /home/kesci/work/similarity_feature/test_idf/consine_sentence_sim.csv | head -n 1
!cat /home/kesci/work/similarity_feature/test_idf/sentence_vector.csv | head -n 1

In [ ]:
########################  query_word & sentence similarity 的计算 ###################
########################          min,max,avg,median,std          ###################
########################  query_word & sentence similarity 的计算 ###################
########################          min,max,avg,median,std          ###################
########################  query_word & sentence similarity 的计算 ###################
########################          min,max,avg,median,std          ###################
########################  query_word & sentence similarity 的计算 ###################
########################          min,max,avg,median,std          ###################

In [ ]:
##########################################废弃代码##############################
##########################################废弃代码##############################
##########################################废弃代码##############################

In [1]:
import pandas as pd
import numpy as np
from numpy import linalg
from sklearn.model_selection import StratifiedKFold
import nltk
import pickle
from nltk import pos_tag
from scipy.sparse import hstack, vstack
from scipy import spatial
from sklearn.datasets import dump_svmlight_file,load_svmlight_file
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
import gensim
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import multiprocessing
import gc
import logging
# 显示cell运行时长
%load_ext klab-autotime

In [2]:
colnames = ["query_id","query","query_title_id","title","label"]

# # save dict
# save_path = "/home/kesci/work/word2vec/wordvectors.kv"
# w2vmodel_test = pickle.load(open(model_path+"word2vec_model_new_withTest.pkl","rb"))
# w2vmodel_test.wv.save(save_path)
save_path = "/home/kesci/work/word2vec/wordvectors.kv"
w2v_dict = KeyedVectors.load(save_path, mmap='r')
len(w2v_dict["1427"])
################ avg word2vec 表征 sentence vector ##############
################ avg word2vec 表征 sentence vector ##############
################ avg word2vec 表征 sentence vector ##############

128

time: 3.49 s


In [8]:
def cosine_sim(query, title):
    l2_query = linalg.norm(query,ord=2,axis=1)
    l2_title = linalg.norm(title,ord=2,axis=1)
    eudistance = linalg.norm(query-title,ord=2,axis=1) # 欧式距离 L2
    manhattan = linalg.norm(query-title,ord=1,axis=1) # 曼哈顿距离 L1
    chebyshev = linalg.norm(query-title,ord=np.inf,axis=1) # 切比雪夫距离 Max|x1-x2|
    chebyshev_ = linalg.norm(query-title,ord=-np.inf,axis=1) # Min |x1-x2|
    my_sqrt_distance = np.sum(np.sqrt(np.absolute(query-title)),axis=1) # sqrt|x1-x2|
    sim = np.sum(query * title,axis=1)/(l2_query * l2_title)
    
    combine_sim = np.hstack((sim.reshape(-1,1),
                             eudistance.reshape(-1,1),
                             manhattan.reshape(-1,1),
                             chebyshev.reshape(-1,1),chebyshev_.reshape(-1,1),
                             my_sqrt_distance.reshape(-1,1)
                            ))
    return combine_sim

#### 1w大小的数据  计算一次
def concat_sentence(origin_data):
    q_vstack = ""
    t_vstack = ""
    for row in origin_data.values:
        # 1.query
        words = row[1].split(" ")
        M = []
        for w in words:
            try:
                M.append(w2v_dict[w])
            except:
                continue
        M = np.array(M)
        if M.shape[0] != 0:
            q_sentence_vec = M.sum(axis=0) / M.shape[0]
        else:
            q_sentence_vec = np.full(word_dim,np.finfo(np.float32).eps)
        if q_vstack == "":
            q_vstack = q_sentence_vec
        else:
             q_vstack = np.vstack((q_vstack,q_sentence_vec))
        
        # 2.title
        words = row[3].split(" ")
        M = []
        for w in words:
            try:
                M.append(w2v_dict[w])
            except:
                continue
        M = np.array(M)
        if M.shape[0] != 0:
            t_sentence_vec = M.sum(axis=0) / M.shape[0]
        else:
            t_sentence_vec = np.full(word_dim,np.finfo(np.float32).eps)
        if t_vstack == "":
            t_vstack = t_sentence_vec
        else:
             t_vstack = np.vstack((t_vstack,t_sentence_vec))
    return q_vstack,t_vstack

time: 7.22 ms


In [9]:
############## 后1000w-500w 验证集数据 ############
feature_path = "/home/kesci/work/similarity_feature/valid/"
train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# sentece vector of (query-title)
sentence_vector_path = feature_path+"sentence_vector.csv"
sentence_vec_fout = open(sentence_vector_path,"w")
# 6dim similarity feature
consine_feature_path = feature_path+"consine_sentence_sim.csv"
sim_qt_fout = open(consine_feature_path,"w")

samples = 100000000
chunksize = 10000
skip_num = int(samples/chunksize) - 1000
print(skip_num)

for i in range(int(500)):
    df = pd.read_csv(train_data, names=colnames, header=None,skiprows=chunksize*(skip_num+i),nrows=chunksize,lineterminator="\n")
    # 1.get sentece_vector & consine sim
    q_vstack,t_vstack = concat_sentence(df)
    combine_sim = cosine_sim(q_vstack,t_vstack)
    # 2.save
    np.savetxt(sentence_vec_fout,q_vstack - t_vstack,delimiter=",",fmt="%.5f")
    np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f")

    # 3.print log
    if (i+1) % 100 == 0:
        print((q_vstack - t_vstack).shape,combine_sim.shape)
    # # 4.gc
    # del df,q_vstack,t_vstack
    # gc.collect()

sentence_vec_fout.flush()
sim_qt_fout.flush()


9000


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
time: 4h 51min 38s


In [10]:
!cat /home/kesci/work/similarity_feature/valid/consine_sentence_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/valid/sentence_vector.csv | wc -l

5000000
5000000
time: 6.69 s


In [ ]:
############################### 废弃代码 #################################
############################### 废弃代码 #################################
############################### 废弃代码 #################################

In [43]:
# from collections import Counter
# model_path = "/home/kesci/work/word2vec/"
# weight_dict = pickle.load(open(model_path+"idfweight+test.pkl","rb")) 
# train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# chunksize =  10000

# def topWord(sentence):
#     tokens = sentence.split(" ")
#     q_cnt = Counter()
#     for word in tokens:
#         q_cnt[word] +=  weight_dict[word]
#     return q_cnt.most_common(1)[0][0]

# count = 0
# key_words = []
# key_word_count = dict()
# ket_word_label = dict()
# for df in pd.read_csv(train_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         key_word = topWord(row[1])
#         key_words.append(key_word)
#         key_word_count[key_word] = key_word_count.get(key_word,0) + 1
#         if row[-1] == 1:
#             ket_word_label[key_word] = ket_word_label.get(key_word,0) + 1
#     if count % 1000 == 0:
#         print(count," ok_1kw")
#     count += 1

# print(len(key_words))

# path = "/home/kesci/work/ctr/"
# pickle.dump(key_words,open(path+"key_words","wb"))
# pickle.dump(key_word_count,open(path+"key_word_count","wb"))
# pickle.dump(ket_word_label,open(path+"ket_word_label","wb"))

0  ok_1kw
1000  ok_1kw
2000  ok_1kw
3000  ok_1kw
4000  ok_1kw
5000  ok_1kw
6000  ok_1kw
7000  ok_1kw
8000  ok_1kw
9000  ok_1kw
100000000
time: 22min 53s


In [44]:
# for key,val in ket_word_label.items():
#     ket_word_label[key] = float(val)/key_word_count[key]
# pickle.dump(ket_word_label,open(path+"key_word_origin_ctr","wb"))

time: 709 ms


In [46]:
# key_word_ctr = pickle.load(open(path+"key_word_origin_ctr","rb"))
# feature_ctr_path = path + "feature_ctr.csv"

# avg = 0.29
# with open(feature_ctr_path,"w") as fout:
#     for word in key_words:
#         ctr = key_word_ctr.get(word,avg)
#         fout.write(str(ctr)+"\n")

# !cat /home/kesci/work/ctr/feature_ctr.csv | wc -l

100000000
time: 2min 44s


In [50]:
# !cat /home/kesci/work/ctr/feature_ctr.csv | head -n 1

0.31769722814498935
cat: write error: Broken pipe
time: 882 ms


In [11]:
# from collections import Counter
# model_path = "/home/kesci/work/word2vec/"
# weight_dict = pickle.load(open(model_path+"idfweight+test.pkl","rb")) 
# test_data = "/home/kesci/input/bytedance/first-round/test.csv"
# chunksize =  10000

# def topWord(sentence):
#     tokens = sentence.split(" ")
#     q_cnt = Counter()
#     for word in tokens:
#         q_cnt[word] +=  weight_dict[word]
#     return q_cnt.most_common(1)

# count = 0
# key_words_unigram_test = []
# for df in pd.read_csv(test_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         key_words_unigram_test.append(topWord(row[1])[0][0])
#     if count % 1000 == 0:
#         print(count," ok_1kw")
#     count += 1

# print(len(key_words_unigram_test))

0  ok_1kw
5000000
time: 1min 12s


In [12]:
# key_words_unigram_test_set = set(key_words_unigram_test)
# print(len(key_words_unigram_test_set))

176124
time: 361 ms


In [6]:
# repeat_q = 0
# for test_q in key_words_unigram_test:
#     if test_q in key_words_unigram_set:
#         repeat_q += 1
        
# print(repeat_q)

# ######### 统计tf_idf 49w的单词能在训练集中出现 (训练集大于50w关键单词  未知是否有良好的区分度)

time: 1.14 ms


In [4]:
# ############ 关键单词的ctr值(观察区分度)
# from collections import Counter
# model_path = "/home/kesci/work/word2vec/"
# weight_dict = pickle.load(open(model_path+"idfweight+test.pkl","rb")) 
# train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# chunksize =  10000

# def topWord(sentence):
#     tokens = sentence.split(" ")
#     q_cnt = Counter()
#     for word in tokens:
#         q_cnt[word] +=  weight_dict[word]
#     return q_cnt.most_common(1)[0][0]

# key_word_count = dict()
# ket_word_label = dict()
# for df in pd.read_csv(train_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         key_word = topWord(row[1])
#         key_word_count[key_word] = key_word_count.get(key_word,0) + 1
#         if row[-1] == 1:
#             ket_word_label[key_word] = ket_word_label.get(key_word,0) + 1

# print(len(key_word_count))
# print(len(ket_word_label))

# for key,val in ket_word_label.items():
#     ket_word_label[key] = val/key_word_count[key]
    

510583
510583
time: 21min 54s


In [33]:
# ########## 置信度 单词出现次数（浏览数）大于一定的阈值
# count = 0
# for key,val in key_word_count.items():
#     if val <= 10:
#         count += 1
# print(count)

185670
time: 104 ms


In [37]:
# ########### 平均ctr
# ctr_all = 0
# count = 0
# for key,val in ket_word_label.items():
#     if key_word_count[key] <= 5:
#         ctr_all += val
#         count += 1
# print(ctr_all/count)


0.3858060100070629
time: 236 ms


In [14]:
# count = 0
# for key,val in ket_word_label.items():
#     if val <= 0.1:
#         count += 1
# print(count)

13769
time: 107 ms


In [5]:
# repeat_q = 0
# repeat_t = 0
# count = 0
# for df in pd.read_csv(test_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     tmp_q = 
#     df['query'].values
#     tmp_t = df['title'].values
#     for q in tmp_q:
#         if q in query_set:
#             repeat_q += 1
            
#     for t in tmp_t:
#         if t in title_set:
#             repeat_t += 1
#     del tmp_q
#     del tmp_t
#     gc.collect()
    
#     print(str(count) + " ok")
#     if count >= 9:
#         break
#     count += 1
    
# print(repeat_q,repeat_t)

time: 462 µs


In [53]:
# ###################  query  长度测试
# count1 = 0
# count2 = 0
# count3 = 0
# count4 = 0
# chunksize = 10000000
# for df in pd.read_csv(train_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         tokens = row[1].split(" ")
#         if len(tokens) == 1:
#             count1 += 1
#         elif len(tokens) == 2:
#             count2 += 1
#         elif len(tokens) == 3:
#             count3 += 1
#         else:
#             count4 += 1
#     print("ok")        
# print(count1,count2,count3,count4)

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
2384672 21517533 28889730 47208065
time: 25min 40s


In [54]:
# ###################  query  长度测试
# test_data = "/home/kesci/input/bytedance/first-round/test.csv"
# count1 = 0
# count2 = 0
# count3 = 0
# count4 = 0
# chunksize = 1000000
# for df in pd.read_csv(test_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         tokens = row[1].split(" ")
#         if len(tokens) == 1:
#             count1 += 1
#         elif len(tokens) == 2:
#             count2 += 1
#         elif len(tokens) == 3:
#             count3 += 1
#         else:
#             count4 += 1
#     print("ok")        
# print(count1,count2,count3,count4)

ok
ok
ok
ok
ok
153582 1020210 1423634 2402574
time: 19.1 s


In [4]:
# ####################### unigram 测试   2019 06 10####################
# ####################### unigram 测试   2019 06 10####################
# ####################### unigram 测试   2019 06 10####################

# from collections import Counter
# colnames = ["query_id","query","query_title_id","title","label"]
# train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# weight_dict = pickle.load(open("/home/kesci/work/ctr/idf_weight_only_train.pkl","rb")) 
# chunksize =  10000
# def topWord(sentence,ngram=1):
#     tokens = sentence.split(" ")
#     q_cnt = Counter()
#     for word in tokens:
#         q_cnt[word] +=  weight_dict[word] # 对于test集中 可能有很多query不在weight_dict
#     keywords = q_cnt.most_common(ngram)
#     return "_".join([word[0] for word in keywords])



time: 584 ms


In [7]:
# key_words_unigram = []
# count = 0
# for df in pd.read_csv(train_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         key_words_unigram.append(topWord(row[1]))
#     if count%1000 == 0:
#         print(count)
#     count += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
time: 25min 19s


In [8]:
# key_words_unigram_set = set(key_words_unigram)
# print(len(key_words_unigram))
# print(len(key_words_unigram_set))


100000000
550485
time: 26.8 s


In [9]:
# from collections import Counter
# colnames = ["query_id","query","query_title_id","title","label"]
# model_path = "/home/kesci/work/word2vec/"
# test_data = "/home/kesci/input/bytedance/first-round/test.csv"
# chunksize =  10000

# def topWord(sentence,ngram=1):
#     tokens = sentence.split(" ")
#     q_cnt = Counter()
#     for word in tokens:
#         q_cnt[word] +=  weight_dict.get(word,1)
#     keywords = q_cnt.most_common(ngram)
#     return "_".join([word[0] for word in keywords])
    
# key_words_unigram_test = []
# count = 0
# for df in pd.read_csv(test_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         key_words_unigram_test.append(topWord(row[1]))
#     if count%100 == 0:
#         print(count)
#     count += 1

0
100
200
300
400
time: 6min 26s


In [13]:
# key_words_unigram_test_set = set(key_words_unigram_test)
# print(len(key_words_unigram_test))
# print(len(key_words_unigram_test_set))

# count = 0
# for q in key_words_unigram_test_set:
#     if q in key_words_unigram_set:
#         count += 1
# print(count)

5000000
176662
173289
time: 702 ms


In [ ]:
############# 训练集计算unigram的点击率特征
for i in ()


In [16]:
# ######################  bigram 测试 ##################
# ######################  bigram 测试 ##################

# from collections import Counter
# weight_dict = pickle.load(open("/home/kesci/work/ctr/idf_weight_only_train.pkl","rb")) 
# def topWord(sentence,ngram=2):
#     tokens = sentence.split(" ")
#     q_cnt = Counter()
#     for word in tokens:
#         q_cnt[word] +=  weight_dict[word] # 对于test集中 可能有很多query不在weight_dict
#     keywords = q_cnt.most_common(ngram)
#     return "_".join([word[0] for word in keywords])
    
# topWord("1")

'1'

time: 3.01 s


In [7]:
# ######################  bigram 测试 ##################
# ######################  bigram 测试 ##################
# from collections import Counter
# colnames = ["query_id","query","query_title_id","title","label"]
# model_path = "/home/kesci/work/word2vec/"
# train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# chunksize =  10000

# key_words_bigram = []
# count = 0
# for df in pd.read_csv(train_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         key_words_bigram.append(topWord(row[1]))
#     if count%1000 == 0:
#         print(count)
#     count += 1


time: 639 µs


In [8]:
# key_words_bigram_set = set(key_words_bigram)
# print(len(key_words_bigram))
# print(len(key_words_bigram_set))

time: 401 µs


In [9]:
# pickle.dump(key_words_bigram_set,open("/home/kesci/work/ctr/key_words_bigram_set.keywords","wb"))
# pickle.dump(key_words_bigram,open("/home/kesci/work/ctr/key_words_bigram.keywords","wb"))

time: 374 µs


In [10]:
# from collections import Counter
# colnames = ["query_id","query","query_title_id","title","label"]
# model_path = "/home/kesci/work/word2vec/"
# test_data = "/home/kesci/input/bytedance/first-round/test.csv"
# chunksize =  10000

# def topWord(sentence,ngram=2):
#     tokens = sentence.split(" ")
#     q_cnt = Counter()
#     for word in tokens:
#         # q_cnt[word] +=  weight_dict[word] # 对于test集中 可能有很多query不在weight_dict当中
#         q_cnt[word] +=  weight_dict.get(word,1)
#     keywords = q_cnt.most_common(ngram)
#     return "_".join([word[0] for word in keywords])
    
# key_words_bigram_test = []
# count = 0
# for df in pd.read_csv(test_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         key_words_bigram_test.append(topWord(row[1]))
#     if count%100 == 0:
#         print(count)
#     count += 1

time: 875 µs


In [51]:
# key_words_bigram_test_set = set(key_words_bigram_test)
# pickle.dump(key_words_bigram_test_set,open("/home/kesci/work/ctr/key_words_bigram_test_set.keywords","wb"))
# pickle.dump(key_words_bigram_test,open("/home/kesci/work/ctr/key_words_bigram_test.keywords","wb"))

time: 2.59 s


5000000
1619257
time: 1.03 s


In [54]:
# print(len(key_words_bigram_test))
# key_words_bigram_test_set = set(key_words_bigram_test)
# print(len(key_words_bigram_test_set))

# count = 0
# double = 0
# for q in key_words_bigram_test_set:
#     words = q.split("_")
#     tmp_q = ""
#     if len(words) == 2:
#         tmp_q = words[1] + "_" + words[0]
#         double += 1
#     if q in key_words_bigram_set or tmp_q in key_words_bigram_set:
#         count += 1

# print(count)
# print(double)

5000000
816096
383996
801342
time: 2.75 s


In [ ]:
# ############ 训练集800w的query关键词的ctr的区分度
# from collections import Counter
# colnames = ["query_id","query","query_title_id","title","label"]
# model_path = "/home/kesci/work/word2vec/"
# train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# chunksize =  10000

# key_words_bigram = []
# count = 0
# for df in pd.read_csv(train_data, names=colnames, header=None,chunksize=chunksize,lineterminator="\n"):
#     for row in df.values:
#         if row[-1] == 1:
            
#     if count%1000 == 0:
#         print(count)
#     count += 1
    
# ##  其他思路 使用余弦相似度寻找最相近的train-query的匹配